In [1]:
from gliner import GLiNER
from contextlib import redirect_stdout
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

c:\Users\abhin\miniconda3\envs\nlpEnv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with redirect_stdout(None):
    model = GLiNER.from_pretrained("urchade/gliner_mediumv2.1")
    model.eval()

c:\Users\abhin\miniconda3\envs\nlpEnv\lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def get_objects(text):
    doc = nlp(text)
    labels = ["ORGANIZATION"]
    entities = model.predict_entities(text, labels, threshold=0.4)
    my_dict = {}

    for ent in entities:
        my_dict.setdefault(f"gliner_{ent['label']}", []).append(ent["text"])
    
    for ent in doc.ents:
        my_dict.setdefault(f"spacy_{ent.label_}", []).append(ent.text)
    
    return pd.Series(my_dict)

In [ ]:
df = pd.read_csv("b2.csv")
df["Sentences"] = df.Sentences.apply(lambda x:x.lstrip().replace(";", ","))
df = pd.concat([df, df.Sentences.apply(get_objects)], axis=1)
df

,Sentences,gliner_ORGANIZATION,spacy_ORG,spacy_PERSON,spacy_DATE,spacy_PRODUCT,spacy_GPE,spacy_LOC,spacy_NORP,spacy_LAW,spacy_TIME,spacy_FAC,spacy_CARDINAL,spacy_WORK_OF_ART
0,"I love my new Nike sneakers, they're so comfor...",[Nike],[Nike],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,McDonald's is my go-to place for a quick burge...,[McDonald's],[McDonald's],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"I need to buy some new Apple products, their l...",[Apple],[Apple],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Coca-Cola is my favorite soft drink, nothing b...",[Coca-Cola],[Coca-Cola],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"I always buy my groceries from Walmart, they h...",[Walmart],[Walmart],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,He finally bought that new drone he's been wan...,[hobby shop],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,We love browsing the latest fashion trends at ...,[Zara],NaN,[Zara],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,She needed some new workout clothes and found ...,[Ross Dress for Less],NaN,[Ross Dress],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,I needed some new light bulbs for my apartment...,[hardware store],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[df.gliner_ORGANIZATION.isna()]

,Sentences,gliner_ORGANIZATION,spacy_ORG,spacy_PERSON,spacy_DATE,spacy_PRODUCT,spacy_GPE,spacy_LOC,spacy_NORP,spacy_LAW,spacy_TIME,spacy_FAC,spacy_CARDINAL,spacy_WORK_OF_ART
45,I always drink Red Bull when I need a pick-me-...,NaN,[Red Bull],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,"I love the new flavors of Lay's Stax chips, th...",NaN,NaN,NaN,NaN,NaN,[Lay],NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,I always drink Monster energy drinks when I ne...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,I love the new flavors of Lay's Poppables chip...,NaN,[Lay's Poppables],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,"I always use Ivory soap, it's gentle on my skin",NaN,NaN,NaN,NaN,NaN,[Ivory],NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,I stocked up on snacks and drinks for the road...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,I finally downloaded that new fitness app I've...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,I finally downloaded that new audiobook I've b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,I needed some new printer ink and found a comp...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,I finally bought that new pair of wireless hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
